In [1]:
import pandas as pd
ebd = pd.read_excel('../data/EBD_new_3.xlsx') 
bd = pd.read_excel('../data/BD_data_all.xlsx') 
print('EBD 컬럼:', ebd.columns.tolist()) 
print('BD 컬럼:', bd.columns.tolist()) 

EBD 컬럼: ['SEQ_NO', 'RECAP_PK', '연면적', '기관명', '건축물명', '주소', '지상', '지하', '사용승인연도', '지역', '용도', '면적', '냉난방면적', '기관유형', '코드번호', '건축물 등록일', '연간 단위면적당 1차 에너지 소비량 3개년 평균', '연간 단위면적당 1차 에너지 소비량 3개년 평균 중간값', '간절기 대비 소비량 비율[(냉방월 +난방월)/간절기월] 3개년 평균', '간절기 대비 소비량 비율[(냉방월 +난방월)/간절기월] 2개년 평균', '간절기 대비 소비량 비율[(냉방월 +난방월)/간절기월] 1개년 평균', '전기 (KWH)', '가스 (KWH)', '지역냉난방 (KWH)', '유류 (KWH)', '기타 (KWH)', '건축물 시설관리자 이름', '건축물 시설관리자  연락처', '건축물 에너지사용량 입력자 이름', '건축물 에너지사용량 입력자 연락처', 'EUI(1차E/냉난방 면적)', 'EUI(1차E/연면적)']
BD 컬럼: ['RECAP_PK', 'MGM_BLD_PK', 'TOTAREA', 'BLD_NM', 'DONG_NM', 'USE_DATE', 'BLD_TYPE_GB_CD', 'MGM_UPPER_BLD_PK', 'REGSTR_GB_CD', 'REGSTR_KIND_CD', 'SIGUNGU_CD', 'BJDONG_CD', 'PLAT_GB_CD', 'BUN', 'JI', 'MAIN_ATCH_GB_CD', 'MAIN_PURPS_CD', 'ETC_PURPS', 'CYEAR', 'BLD_N', 'MULTI_YN', 'CYEAR_CD']


In [2]:
ebd['SEQ_NO'].nunique()

4194

In [10]:

# 가정:
# ebd: EBD 데이터프레임 (컬럼: 'RECAP_PK', '연면적', '사용승인연도', ...)
# bd: BD 데이터프레임 (컬럼: 'RECAP_PK', 'TOTAREA', 'USE_DATE', 'MGM_BLD_PK', ...)

# 비교할 범위(퍼센트)를 딕셔너리로 정의합니다.
thresholds = {
    "±1%": 0.01,
    "±5%": 0.05,
    "±10%": 0.10,
    "±20%": 0.20
}

def count_bd_matches(row):
    """
    각 EBD 행(row)에 대해 같은 RECAP_PK를 가진 BD 데이터 중에서,
    EBD의 '연면적'을 기준으로 BD의 'TOTAREA'가 지정된 범위 내에 있는 행 수를 계산합니다.
    결과는 각 범위별 건수를 담은 Series로 반환합니다.
    """
    ebd_area = row["연면적"]
    recap = row["RECAP_PK"]
    # 해당 RECAP_PK를 가진 BD 행들을 필터링
    bd_subset = bd[bd["RECAP_PK"] == recap]
    
    # 각 범위별로 BD 행 수를 계산
    counts = {}
    for label, pct in thresholds.items():
        lower_bound = ebd_area * (1 - pct)
        upper_bound = ebd_area * (1 + pct)
        count = bd_subset[(bd_subset["TOTAREA"] >= lower_bound) & (bd_subset["TOTAREA"] <= upper_bound)].shape[0]
        counts[label] = count
    return pd.Series(counts)

# 각 EBD 행마다 BD 매칭 건수를 계산 (결과는 EBD 행과 동일한 인덱스, 각 범위별 건수를 담은 DataFrame)
ebd_bd_counts = ebd.apply(count_bd_matches, axis=1)

# 예를 들어, 전체 EBD 데이터에 대해 집계한 총 건수 (각 범위별 BD 매칭 건수의 합)를 확인할 수 있습니다.
total_counts = ebd_bd_counts.sum()
print("EBD 건물 기준, 동일 RECAP의 BD 데이터 중 면적 범위 내에 포함되는 BD 행 건수:")
print(total_counts)

# 또는 각 EBD 행에 대해 개별 결과를 ebd 데이터와 결합해서 확인할 수도 있습니다.
ebd_with_counts = pd.concat([ebd, ebd_bd_counts], axis=1)
print(ebd_with_counts.head())


EBD 건물 기준, 동일 RECAP의 BD 데이터 중 면적 범위 내에 포함되는 BD 행 건수:
±1%      2977
±5%      4730
±10%     6578
±20%    10131
dtype: int64
   SEQ_NO   RECAP_PK       연면적  기관명             건축물명                      주소  \
0     972  11110-900  10787.40  감사원               본관         서울특별시 종로구 25-23   
1     974  11110-900   5458.12  감사원        제2별관(제5호)         서울특별시 종로구 25-23   
2     975  11110-900   4949.30  감사원   제3별관(감사원 제2별관)    서울특별시 종로구 삼청동 28-102   
3     976  41480-189  10615.97  감사원  감사교육원(감사교육원 청사)                 경기도 파주시   
4    1861        NaN   4948.93  교육부          대한민국학술원  서울특별시 서초구 반포동 94-4 학술원   

    지상   지하      사용승인연도     지역  ... 건축물 시설관리자 이름 건축물 시설관리자  연락처  \
0  8.0  2.0  1971-04-21  중부2지역  ...          정준혁   02-2011-3381   
1  2.0  3.0  1991-07-22  중부2지역  ...          정준혁   02-2011-3381   
2  3.0  2.0  2004-11-10  중부2지역  ...          정준혁   02-2011-3811   
3  4.0  1.0  1999-08-26  중부1지역  ...          박원호   031-940-8817   
4  3.0  1.0  1987-11-09  중부2지역  ...          윤성신   02-3400-521

In [3]:
merged = pd.merge(ebd, bd, on='RECAP_PK', how='left', suffixes=('_ebd', '_bd'))
merged

,SEQ_NO,RECAP_PK,연면적,기관명,건축물명,주소,지상,지하,사용승인연도,지역,...,PLAT_GB_CD,BUN,JI,MAIN_ATCH_GB_CD,MAIN_PURPS_CD,ETC_PURPS,CYEAR,BLD_N,MULTI_YN,CYEAR_CD
0,972,11110-900,10787.4,감사원,본관,서울특별시 종로구 25-23,8.0,2.0,1971-04-21,중부2지역,...,0.0,25.0,23.0,0.0,14000,공공업무시설,NaN,5.0,Y,2014.0
1,972,11110-900,10787.4,감사원,본관,서울특별시 종로구 25-23,8.0,2.0,1971-04-21,중부2지역,...,0.0,25.0,23.0,0.0,14000,공공업무시설,NaN,5.0,Y,2014.0
2,972,11110-900,10787.4,감사원,본관,서울특별시 종로구 25-23,8.0,2.0,1971-04-21,중부2지역,...,0.0,25.0,23.0,0.0,14000,공공업무시설,2024.0,5.0,Y,2024.0
3,972,11110-900,10787.4,감사원,본관,서울특별시 종로구 25-23,8.0,2.0,1971-04-21,중부2지역,...,0.0,25.0,23.0,0.0,03000,공용시설,NaN,5.0,Y,2014.0
4,972,11110-900,10787.4,감사원,본관,서울특별시 종로구 25-23,8.0,2.0,1971-04-21,중부2지역,...,0.0,25.0,23.0,0.0,04000,식당,NaN,5.0,Y,2014.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69524,10000006164722,36110-1000000000000000571160,31242.0,정부청사관리본부 세종청사,정부세종청사 2동,"세종특별자치도 세종특별자치시 (어진동 3-187번지, 정부세종청사 2동 )",5.0,1.0,2012-12-17,중부2지역,...,0.0,572.0,0.0,0.0,14000,업무시설(공공청사),2022.0,1.0,N,2022.0
69525,10000006164723,36110-1000000000000000571160,4926.0,정부청사관리본부 세종청사,정부세종청사 3동,"세종특별자치도 세종특별자치시 (어진동 3-187번지, 정부세종청사 3동 )",5.0,1.0,2012-12-17,중부2지역,...,0.0,572.0,0.0,0.0,14000,업무시설(공공청사),2022.0,1.0,N,2022.0
69526,10000006164724,36110-1000000000000000571160,40968.0,정부청사관리본부 세종청사,정부세종청사 4동,"세종특별자치도 세종특별자치시 (어진동 3-187번지, 정부세종청사 4동 )",6.0,1.0,2012-12-17,중부2지역,...,0.0,572.0,0.0,0.0,14000,업무시설(공공청사),2022.0,1.0,N,2022.0
69527,10000006164725,36110-1000000000000000571160,54296.0,정부청사관리본부 세종청사,정부세종청사 5동,"세종특별자치도 세종특별자치시 (어진동 3-187번지, 정부세종청사 5동 )",7.0,1.0,2012-12-17,중부2지역,...,0.0,572.0,0.0,0.0,14000,업무시설(공공청사),2022.0,1.0,N,2022.0


In [3]:
rule_result = pd.read_excel('../result/rule_matching_result_ver3.xlsx') 
rule_result.head()

,SEQ_NO,RECAP_PK,연면적,사용승인연도,기관명,건축물명,주소,지상,지하,TOTAREA,BLD_NM,DONG_NM,USE_DATE,MGM_BLD_PK,MATCH_STAGE,EBD_COUNT,BD_COUNT,EBD_OVER_BD
0,972,11110-900,10787.40,1971-04-21,감사원,본관,서울특별시 종로구 25-23,8.0,2.0,10787.40,본관,NaN,NaN,11110-2450,2차,3.0,5.0,no
1,974,11110-900,5458.12,1991-07-22,감사원,제2별관(제5호),서울특별시 종로구 25-23,2.0,3.0,5458.12,(제5호),NaN,NaN,11110-2449,2차,3.0,5.0,no
2,975,11110-900,4949.30,2004-11-10,감사원,제3별관(감사원 제2별관),서울특별시 종로구 삼청동 28-102,3.0,2.0,NaN,NaN,NaN,NaN,NaN,미매칭,3.0,5.0,no
3,976,41480-189,10615.97,1999-08-26,감사원,감사교육원(감사교육원 청사),경기도 파주시,4.0,1.0,10615.97,감사교육원 청사(본관동),NaN,1999-08-26,41480-6265,1차,1.0,4.0,no
4,1861,NaN,4948.93,1987-11-09,교육부,대한민국학술원,서울특별시 서초구 반포동 94-4 학술원,3.0,1.0,NaN,NaN,NaN,NaN,NaN,미매칭,NaN,NaN,NaN


In [4]:
rule_result['SEQ_NO'].nunique()

4194

In [6]:
columns_to_select = ['SEQ_NO', 'RECAP_PK', '연면적', '사용승인연도', '기관명', '건축물명', '주소', '지상', '지하', 'MGM_BLD_PK', 'MATCH_STAGE']
ebd_df = rule_result[columns_to_select]
ebd_df

,SEQ_NO,RECAP_PK,연면적,사용승인연도,기관명,건축물명,주소,지상,지하,MGM_BLD_PK,MATCH_STAGE
0,972,11110-900,10787.40,1971-04-21,감사원,본관,서울특별시 종로구 25-23,8.0,2.0,11110-2450,2차
1,974,11110-900,5458.12,1991-07-22,감사원,제2별관(제5호),서울특별시 종로구 25-23,2.0,3.0,11110-2449,2차
2,975,11110-900,4949.30,2004-11-10,감사원,제3별관(감사원 제2별관),서울특별시 종로구 삼청동 28-102,3.0,2.0,NaN,미매칭
3,976,41480-189,10615.97,1999-08-26,감사원,감사교육원(감사교육원 청사),경기도 파주시,4.0,1.0,41480-6265,1차
4,1861,NaN,4948.93,1987-11-09,교육부,대한민국학술원,서울특별시 서초구 반포동 94-4 학술원,3.0,1.0,NaN,미매칭
...,...,...,...,...,...,...,...,...,...,...,...
4189,10000006164722,36110-1000000000000000571160,31242.00,2012-12-17,정부청사관리본부 세종청사,정부세종청사 2동,"세종특별자치도 세종특별자치시 (어진동 3-187번지, 정부세종청사 2동 )",5.0,1.0,NaN,미매칭
4190,10000006164723,36110-1000000000000000571160,4926.00,2012-12-17,정부청사관리본부 세종청사,정부세종청사 3동,"세종특별자치도 세종특별자치시 (어진동 3-187번지, 정부세종청사 3동 )",5.0,1.0,NaN,미매칭
4191,10000006164724,36110-1000000000000000571160,40968.00,2012-12-17,정부청사관리본부 세종청사,정부세종청사 4동,"세종특별자치도 세종특별자치시 (어진동 3-187번지, 정부세종청사 4동 )",6.0,1.0,NaN,미매칭
4192,10000006164725,36110-1000000000000000571160,54296.00,2012-12-17,정부청사관리본부 세종청사,정부세종청사 5동,"세종특별자치도 세종특별자치시 (어진동 3-187번지, 정부세종청사 5동 )",7.0,1.0,NaN,미매칭


In [11]:
import pandas as pd

# 예시: ebd_with_counts는 원래 ebd 데이터에 count_bd_matches 결과(예: "±1%", "±5%", "±10%", "±20%" 컬럼)가 붙은 DataFrame입니다.
# final_matches는 최종 매칭 결과 DataFrame으로, 최소 SEQ_NO와 match_stage 컬럼을 포함합니다.

# 예시: ebd_with_counts와 final_matches가 다음과 같이 구성되었다고 가정합니다.
# ebd_with_counts: SEQ_NO, RECAP_PK, 연면적, 사용승인연도, ..., "±1%", "±5%", "±10%", "±20%"
# final_matches: SEQ_NO, RECAP_PK, 연면적, 사용승인연도, TOTAREA, USE_DATE, MGM_BLD_PK, match_stage

# 먼저, 두 DataFrame을 SEQ_NO 기준으로 병합합니다.
merged_with_counts = pd.merge(ebd_df[['SEQ_NO', 'MATCH_STAGE']], 
                                ebd_with_counts[['SEQ_NO', "±1%", "±5%", "±10%", "±20%"]],
                                on='SEQ_NO', how='left')

# MATCH_STAGE가 '미매칭'인 경우만 필터링
mismatched = merged_with_counts[merged_with_counts['MATCH_STAGE'] == '미매칭']

# 각 범위별 BD 매칭 건수를 집계 (각 EBD 행의 값의 합계를 계산)
threshold_columns = ["±1%", "±5%", "±10%", "±20%"]
total_mismatches = mismatched[threshold_columns].sum()

print("MATCH_STAGE가 '미매칭'인 EBD 건물에서 각 면적 범위에 해당하는 BD 매칭 건수:")
print(total_mismatches)


MATCH_STAGE가 '미매칭'인 EBD 건물에서 각 면적 범위에 해당하는 BD 매칭 건수:
±1%      737
±5%     1539
±10%    2280
±20%    3733
dtype: int64


In [11]:
import pandas as pd

# 가정:
# - ebd_with_counts: 원본 EBD 데이터에 count_bd_matches 함수 결과가 붙은 DataFrame
#   이 DataFrame은 최소한 다음 컬럼들을 포함합니다:
#      'SEQ_NO', 'RECAP_PK', '연면적', '사용승인연도', "±1%", "±5%", "±10%", "±20%"
#
# - final_matches: 최종 매칭 결과 DataFrame으로, 최소 'SEQ_NO'와 'match_stage' 컬럼을 포함합니다.
#   여기서 'match_stage'가 '미매칭'인 경우에 해당하는 EBD 행만 별도로 집계합니다.
#
# 먼저, 두 DataFrame을 SEQ_NO를 기준으로 병합하여, 각 EBD 행의 매칭 단계와 면적 범위별 BD 후보 개수를 모두 포함하도록 합니다.
merged_with_counts = pd.merge(ebd_df[['SEQ_NO', 'MATCH_STAGE']], 
                              ebd_with_counts[['SEQ_NO', "±1%", "±5%", "±10%", "±20%"]],
                              on='SEQ_NO', how='left')

# 미매칭인 EBD 행만 선택합니다.
mismatched = merged_with_counts[merged_with_counts['MATCH_STAGE'] == '미매칭']

# 각 범위별로, 해당 EBD 행에서 BD 후보 개수가 1개 이상이면 해당 EBD 건을 "해당 범위에 속한다"고 보고 집계합니다.
threshold_columns = ["±1%", "±5%", "±10%", "±20%"]
ebd_range_counts = {}

for col in threshold_columns:
    ebd_range_counts[col] = (mismatched[col] > 0).sum()

print("미매칭인 EBD 행 중, 각 면적 범위에 해당하는 BD 후보가 존재하는 EBD 건수:")
for label, count in ebd_range_counts.items():
    print(f"{label}: {count} 건")


미매칭인 EBD 행 중, 각 면적 범위에 해당하는 BD 후보가 존재하는 EBD 건수:
±1%: 643 건
±5%: 852 건
±10%: 942 건
±20%: 1039 건


In [9]:
import pandas as pd

def count_exact_matches(row):
    """
    각 EBD 행(row)에 대해 동일 RECAP_PK를 가진 BD 데이터 중,
    BD의 TOTAREA가 EBD의 연면적과 정확히 일치하는 건수를 계산합니다.
    """
    ebd_area = row["연면적"]
    recap = row["RECAP_PK"]
    # 해당 RECAP_PK를 가진 BD 행들을 필터링
    bd_subset = bd[bd["RECAP_PK"] == recap]
    # BD의 TOTAREA가 EBD의 연면적과 정확히 일치하는 행의 개수
    exact_count = bd_subset[bd_subset["TOTAREA"] == ebd_area].shape[0]
    return exact_count

# 원본 EBD 데이터에 대해 각 건물별 정확히 일치하는 BD 후보 개수를 계산한 결과를 새 컬럼으로 추가합니다.
ebd_with_exact = ebd_df.copy()
ebd_with_exact['exact'] = ebd_with_exact.apply(count_exact_matches, axis=1)

# 이미 최종 매칭 결과가 담긴 DataFrame (final_matches)에는 각 EBD 행에 대한 매칭 단계(match_stage)가 포함되어 있다고 가정합니다.
# 여기서 SEQ_NO (또는 고유 식별자)를 기준으로 ebd_with_exact와 final_matches를 병합합니다.
merged_exact = pd.merge(ebd_df[['SEQ_NO', 'MATCH_STAGE']],
                          ebd_with_exact[['SEQ_NO', 'exact']],
                          on='SEQ_NO', how='left')

# 이제, MATCH_STAGE가 '미매칭'인 EBD 행 중에서, 정확히 일치하는 BD 후보가 1건 이상인 경우를 집계합니다.
mismatched_exact = merged_exact[merged_exact['MATCH_STAGE'] == '미매칭']
exact_ebd_count = (mismatched_exact['exact'] > 0).sum()

print("미매칭인 EBD 건 중, BD의 TOTAREA가 정확히 일치하는 경우가 있는 건수:", exact_ebd_count)


미매칭인 EBD 건 중, BD의 TOTAREA가 정확히 일치하는 경우가 있는 건수: 0


1~2차에서 매칭했기 때문에 0건이여야됨. 확인 완료